In [1]:
import heeps
import numpy as np
from astropy.io import fits

In [2]:
def get_psfs(conf, case='scao_only', modes=['RAVC', 'APPIMG', 'CVC', 'CLC']):
    conf.update(
        mode = 'ELT',
        f_phase = 'wavefront/cfull/cube_Cfull_20220512_3600s_300ms_0piston_meters_%s'%case + '_L_285.fits',
        apo_drift = 0.02, # (% ptv)
        f_amp = 'wavefront/talbot/DynamicV08/Talbot_LM_20220223' + '_%s.fits'
            %('_rep_1_field_50' if 'static' in case else 'rep_1_-0h30'),# else rep),
        vc_chrom_leak = 2e-3,
        avg = True,
        hfov = 1.065,
        ngrid = 2048,
    )
    conf = heeps.config.read_config(**conf)
    conf['band_specs']['L']['pscale'] = 5.47/2
    conf = heeps.config.update_config(**conf)
    wf = heeps.pupil.pupil(**conf)
    psf = heeps.wavefront.propagate(wf, **conf)
    elt_peak = np.max(psf)
    print('ELT peak = %.4f'%elt_peak)
    psf /= elt_peak
    fits.writeto(conf['dir_output'] + 'onaxis_PSF_L_ELT.fits', psf, overwrite=True)
    
    verbose = False
    for conf['mode'] in modes:
        if 'RAVC' in conf['mode']:
            conf.update(ls_dRext=0.0477, ls_dRint=0.04, ls_dRspi=0.0249)
        elif 'CVC' in conf['mode']:
            conf.update(ls_dRext=0.0209, ls_dRint=0.09, ls_dRspi=0.0245)
        elif 'CLC' in conf['mode']:
            conf.update(ls_dRext=0.1095, ls_dRint=0.0491, ls_dRspi=0.0317)
        conf = heeps.config.update_config(**conf, verbose=verbose)
        wf = heeps.pupil.pupil(**conf)
        if 'VC' in conf['mode']:
            psf_OFF = heeps.wavefront.propagate(wf, onaxis=False, **conf)
            psf_ON = heeps.wavefront.propagate(wf, onaxis=True, **conf)
            print('mode = %s, leakage = %.2e'%(conf['mode'], np.sum(psf_ON)/np.sum(psf_OFF)))
        else:
            psf_ON = heeps.wavefront.propagate(wf, onaxis=True, **conf)
        psf_ON /= elt_peak
        fits.writeto(conf['dir_output'] + 'onaxis_PSF_L_%s.fits'%conf['mode'], psf_ON, overwrite=True)

### ideal

In [3]:
conf = dict(
    dir_output = 'output_files/instant_psf/ideal',
    nframes_avg = 1,
    add_phase = False,
)
get_psfs(conf)

   2022-07-22 19:01:38, e2e simulation using 56 cores
   2022-07-22 19:01:42, completed in 4.02 seconds
ELT peak = 0.0122
   2022-07-22 19:01:44, e2e simulation using 56 cores
   2022-07-22 19:01:49, completed in 4.76 seconds
   2022-07-22 19:01:50, e2e simulation using 56 cores
   2022-07-22 19:02:01, completed in 10.64 seconds
mode = RAVC, leakage = 2.15e-03
   2022-07-22 19:02:01, e2e simulation using 56 cores
   2022-07-22 19:02:06, completed in 4.71 seconds
   2022-07-22 19:02:08, e2e simulation using 56 cores
   2022-07-22 19:02:13, completed in 5.08 seconds
   2022-07-22 19:02:15, e2e simulation using 56 cores
   2022-07-22 19:02:25, completed in 9.93 seconds
mode = CVC, leakage = 2.59e-02
   2022-07-22 19:02:26, e2e simulation using 56 cores
   2022-07-22 19:02:36, completed in 10.28 seconds


### scao only

In [4]:
conf = dict(
    dir_output = 'output_files/instant_psf/scao_only',
    nframes_avg = 1,
    add_phase = True,
)
get_psfs(conf)

   2022-07-22 19:02:37, e2e simulation using 56 cores
   2022-07-22 19:02:42, completed in 4.46 seconds
ELT peak = 0.0118
   2022-07-22 19:02:45, e2e simulation using 56 cores
   2022-07-22 19:02:50, completed in 5.28 seconds
   2022-07-22 19:02:52, e2e simulation using 56 cores
   2022-07-22 19:03:02, completed in 9.59 seconds
mode = RAVC, leakage = 2.36e-02
   2022-07-22 19:03:03, e2e simulation using 56 cores
   2022-07-22 19:03:08, completed in 4.66 seconds
   2022-07-22 19:03:10, e2e simulation using 56 cores
   2022-07-22 19:03:16, completed in 5.35 seconds
   2022-07-22 19:03:18, e2e simulation using 56 cores
   2022-07-22 19:03:28, completed in 10.51 seconds
mode = CVC, leakage = 4.71e-02
   2022-07-22 19:03:31, e2e simulation using 56 cores
   2022-07-22 19:03:40, completed in 9.58 seconds


### long exposure

In [5]:
conf = dict(
    dir_output = 'output_files/instant_psf/long_exp',
    nframes_avg = 100,
    add_phase = True,
)
get_psfs(conf)

   2022-07-22 19:03:41, e2e simulation using 56 cores
   2022-07-22 19:04:10, completed in 28.5 seconds
ELT peak = 0.0118
   2022-07-22 19:04:13, e2e simulation using 56 cores
   2022-07-22 19:05:11, completed in 58.06 seconds
   2022-07-22 19:05:15, e2e simulation using 56 cores
   2022-07-22 19:06:26, completed in 70.6 seconds
mode = RAVC, leakage = 2.43e-02
   2022-07-22 19:06:28, e2e simulation using 56 cores
   2022-07-22 19:07:21, completed in 52.92 seconds
   2022-07-22 19:07:25, e2e simulation using 56 cores
   2022-07-22 19:08:15, completed in 49.95 seconds
   2022-07-22 19:08:19, e2e simulation using 56 cores
   2022-07-22 19:09:30, completed in 70.59 seconds
mode = CVC, leakage = 4.74e-02
   2022-07-22 19:09:34, e2e simulation using 56 cores
   2022-07-22 19:10:41, completed in 67.45 seconds


In [6]:
conf = dict(
    dir_output = 'output_files/instant_psf/long_exp_AE',
    nframes_avg = 100,
    f_pupil = 'pupil/ELT_fullM1_refl_0.02rms.fits',
    add_phase = True,
    add_apo_drift = True,
    add_amp = True,
    add_cl_vort = True,
)
get_psfs(conf, case='all_ncpa', modes=['RAVC'])

   2022-07-22 19:10:49, e2e simulation using 56 cores
   2022-07-22 19:11:18, completed in 28.67 seconds
ELT peak = 0.0114
   2022-07-22 19:11:28, e2e simulation using 56 cores
   2022-07-22 19:12:18, completed in 50.34 seconds
   2022-07-22 19:12:27, e2e simulation using 56 cores
   2022-07-22 19:13:33, completed in 65.99 seconds
mode = RAVC, leakage = 3.54e-02


In [7]:
conf = dict(
    dir_output = 'output_files/instant_psf/long_exp_AE_misseg',
    nframes_avg = 100,
    f_pupil = 'pupil/ELT_fullM1_refl_0.02rms_misseg_1x7.fits',
    add_phase = True,
    add_apo_drift = True,
    add_amp = True,
    add_cl_vort = True,
)
get_psfs(conf, case='all_ncpa', modes=['RAVC'])

   2022-07-22 19:13:41, e2e simulation using 56 cores
   2022-07-22 19:14:09, completed in 27.97 seconds
ELT peak = 0.0113
   2022-07-22 19:14:19, e2e simulation using 56 cores
   2022-07-22 19:15:09, completed in 50.95 seconds
   2022-07-22 19:15:19, e2e simulation using 56 cores
   2022-07-22 19:16:29, completed in 69.65 seconds
mode = RAVC, leakage = 4.81e-02
